In [ ]:
import satellite_images_nso.api.nso_georegion as nso
import getpass

# Search for satellite images in geojson and download links

In [ ]:
# get credentials
nso_username = getpass.getpass(prompt='username:')
nso_password = getpass.getpass(prompt='password:')

# give path to geojson en output
path_geojson = "C:/Repos/satelliet_nso/Meijendel/Meijendel & Berkheide_wgs84.geojson"
output_path = "C:/Repos/satelliet_nso/output_pip_package/"

# This method fetches all the download links to all the satelliet images which contain region in the geojson
georegion = nso.nso_georegion(path_geojson,output_path,\
                              nso_username,\
                            nso_password)

In [ ]:
links = georegion.retrieve_download_links(max_diff=0.2)

In [ ]:
links

# Download, Crop and calculate NDVI for a LandSat Image

### For a specific image

In [ ]:
# Downloads a satellite image from the NSO, makes a crop out of it so it fits the geojson region and calculates the NVDI index.
# The output will stored in the designated output folder.
# image = links[0]
image = 'https://api.satellietdataportaal.nl/v1/download/SV_RD_11bit_RGBI_50cm/20220501_111418_SV1-03'
georegion.execute_link(image)

# The parameters are : execute_link(self, link, delete_zip_file = False,delete_source_files = True, 
# check_if_file_exists = True,plot=True)

### For a group of images 

In [ ]:
# loop through and download all the 200cm resolution SuperView images
nr_links=0
for link in links:
    if 'SV' in link and '50cm' in link and 'RGBI' in link: #and '2022' in link
        print(link)
        nr_links+= 1
        try:
            georegion.execute_link(link)
        except Exception as e:
            print(e)
print(nr_links)

In [ ]:
# loop through and download all the 200cm Trisat images
for link in links:
    if 'Tri' in link and "80cm" in link and "8bit" in link:
        try:
            georegion.execute_link(link)
        except Exception as e:
            print(e)

# Normalize LandSat image

### Multi date relative normalisation.

In [ ]:
# Imports the normalisation library.
from satellite_images_nso.__normalisation import normalisation

# Path to a Superview satellite image.
path = "20190401_111723_SV1-04_200cm_RD_11bit_RGBI_DenHaag_solleveld_sweco_cropped.tif"

normalisation.multidate_normalisation_75th_percentile(path)

### Black spot normalisation.

In [ ]:
path = "data_satellite/20210302_111341_SV1-03_SV_RD_11bit_RGBI_200cm_Brouwersdam_Grevelingen_21_cropped.tif"

In [ ]:
# The first argument here is the path to satellite image and the second is the name of the satellite image to get coefficients for, this can be filled in partially.
normalisation.multi_date_dark_spot_normalisation(path,"20210302")

# Other functionalities

In [ ]:
# The sat_manipulator gives other handy transmations on satellite data.
import satellite_images_nso.api.sat_manipulator as sat_manipulator

# This function reads a .tif file, which is a format the satellite data is stored in,  and converts it to a pixel based geopandas dataframe.
# For machine learning purposes.
path_to_vector = "path/to/folder/*.tif"
geo_df_pixel = sat_manipulator.tranform_vector_to_pixel_gpdf(path_to_vector)